In [1]:
import glob
import pandas as pd

training_pathfiles = glob.glob("/Users/tedlinghu/Desktop/Luo_Rotation/luorotation/training_data/*.csv")

testing_pathfiles = glob.glob("/Users/tedlinghu/Desktop/Luo_Rotation/luorotation/testing_data/*.csv")


In [2]:
training_dict = dict()
for path in training_pathfiles:
    training_dict[path.split("vars_")[1].split(".csv")[0]] = pd.read_csv(path)

In [3]:
testing_dict = dict()
for path in testing_pathfiles:
    testing_dict[path.split("vars_")[1].split(".csv")[0]] = pd.read_csv(path)
    

In [4]:
import random
validation_dict = dict()
training_label_dict = dict()
validation_label_dict = dict()

toreplace = {"ALIVE" : 1, "EXPIRED":0 }


for i in training_dict:
    training_dict[i] = training_dict[i].drop('Unnamed: 0', axis = 1)
    idx = random.sample(range(len(training_dict[i])), int(training_dict[i].shape[0]*0.2))
    validation_dict[i] = training_dict[i].loc[idx]
    validation_label_dict[i] = validation_dict[i]['label']
    validation_label_dict[i].replace(toreplace, inplace = True)
    validation_dict[i] = validation_dict[i].drop('label', axis = 1)
    
    training_dict[i] = training_dict[i].drop(idx)
    training_label_dict[i] = training_dict[i]['label']
    training_label_dict[i].replace(toreplace, inplace = True)
    training_dict[i] = training_dict[i].drop('label', axis = 1)

In [5]:
print(training_dict['div_downsampled_training'].shape)
print(validation_dict['div_downsampled_training'].shape)
print(training_label_dict['div_downsampled_training'].shape)
print(validation_label_dict['div_downsampled_training'].shape)


(2721, 85)
(680, 85)
(2721,)
(680,)


In [8]:
from sklearn.model_selection import GridSearchCV
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier

lda = LinearDiscriminantAnalysis()
rf = RandomForestClassifier()


sk_models = [lda, rf]
sk_model_names = ["lda", "rf"]


#Training
models_dict = dict()

for m in range(len(sk_models)):
    for d in training_dict:
        models_dict[sk_model_names[m] + "_" + d] = sk_models[m].fit(training_dict[d] , training_label_dict[d])


#validation
grid_search_dict = dict()
rf_param_grid = [
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
  ]

lda_param_grid = [
    {'solver':['svd', 'lsqr'] , 'n_components': [1,2,3,4,5, 20, 50, 100]},
  ]


param_grid = [lda_param_grid, rf_param_grid]

for i in range(len(sk_models)):
    for d in validation_dict:
        grid_search = GridSearchCV(sk_models[i], param_grid[i], cv=5,
                                   scoring='roc_auc',
                                   return_train_score=True)
        grid_search_dict[sk_model_names[i] + "_" + d] = grid_search.fit(validation_dict[d] , validation_label_dict[d])



/Users/tedlinghu/opt/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/Users/tedlinghu/opt/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/Users/tedlinghu/opt/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/Users/tedlinghu/opt/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/Users/tedlinghu/opt/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/Users/tedlinghu/opt/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:388: UserWarn

/Users/tedlinghu/opt/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/Users/tedlinghu/opt/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:466: ChangedBehaviorWarning: n_components cannot be larger than min(n_features, n_classes - 1). Using min(n_features, n_classes - 1) = min(85, 2 - 1) = 1 components.
  ChangedBehaviorWarning)
/Users/tedlinghu/opt/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:472: FutureWarning: In version 0.23, setting n_components > min(n_features, n_classes - 1) will raise a ValueError. You should set n_components to None (default), or a value smaller or equal to min(n_features, n_classes - 1).
  warnings.warn(future_msg, FutureWarning)
/Users/tedlinghu/opt/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.

/Users/tedlinghu/opt/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:466: ChangedBehaviorWarning: n_components cannot be larger than min(n_features, n_classes - 1). Using min(n_features, n_classes - 1) = min(85, 2 - 1) = 1 components.
  ChangedBehaviorWarning)
/Users/tedlinghu/opt/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:472: FutureWarning: In version 0.23, setting n_components > min(n_features, n_classes - 1) will raise a ValueError. You should set n_components to None (default), or a value smaller or equal to min(n_features, n_classes - 1).
  warnings.warn(future_msg, FutureWarning)
/Users/tedlinghu/opt/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:466: ChangedBehaviorWarning: n_components cannot be larger than min(n_features, n_classes - 1). Using min(n_features, n_classes - 1) = min(85, 2 - 1) = 1 components.
  ChangedBehaviorWarning)
/Users/tedlinghu/opt/anaconda3/lib/python3.7/site-packages/sklearn/disc

/Users/tedlinghu/opt/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:466: ChangedBehaviorWarning: n_components cannot be larger than min(n_features, n_classes - 1). Using min(n_features, n_classes - 1) = min(85, 2 - 1) = 1 components.
  ChangedBehaviorWarning)
/Users/tedlinghu/opt/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:472: FutureWarning: In version 0.23, setting n_components > min(n_features, n_classes - 1) will raise a ValueError. You should set n_components to None (default), or a value smaller or equal to min(n_features, n_classes - 1).
  warnings.warn(future_msg, FutureWarning)
/Users/tedlinghu/opt/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:466: ChangedBehaviorWarning: n_components cannot be larger than min(n_features, n_classes - 1). Using min(n_features, n_classes - 1) = min(85, 2 - 1) = 1 components.
  ChangedBehaviorWarning)
/Users/tedlinghu/opt/anaconda3/lib/python3.7/site-packages/sklearn/disc

/Users/tedlinghu/opt/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/Users/tedlinghu/opt/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/Users/tedlinghu/opt/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/Users/tedlinghu/opt/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/Users/tedlinghu/opt/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/Users/tedlinghu/opt/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:466: ChangedB

...

/Users/tedlinghu/opt/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:466: ChangedBehaviorWarning: n_components cannot be larger than min(n_features, n_classes - 1). Using min(n_features, n_classes - 1) = min(17, 2 - 1) = 1 components.
  ChangedBehaviorWarning)
/Users/tedlinghu/opt/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:472: FutureWarning: In version 0.23, setting n_components > min(n_features, n_classes - 1) will raise a ValueError. You should set n_components to None (default), or a value smaller or equal to min(n_features, n_classes - 1).
  warnings.warn(future_msg, FutureWarning)
/Users/tedlinghu/opt/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/Users/tedlinghu/opt/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.

/Users/tedlinghu/opt/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:466: ChangedBehaviorWarning: n_components cannot be larger than min(n_features, n_classes - 1). Using min(n_features, n_classes - 1) = min(289, 2 - 1) = 1 components.
  ChangedBehaviorWarning)
/Users/tedlinghu/opt/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:472: FutureWarning: In version 0.23, setting n_components > min(n_features, n_classes - 1) will raise a ValueError. You should set n_components to None (default), or a value smaller or equal to min(n_features, n_classes - 1).
  warnings.warn(future_msg, FutureWarning)
/Users/tedlinghu/opt/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/Users/tedlinghu/opt/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:466: ChangedBehaviorWarning: n_components cannot be larger than min(n_features, n_c

/Users/tedlinghu/opt/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:466: ChangedBehaviorWarning: n_components cannot be larger than min(n_features, n_classes - 1). Using min(n_features, n_classes - 1) = min(289, 2 - 1) = 1 components.
  ChangedBehaviorWarning)
/Users/tedlinghu/opt/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:472: FutureWarning: In version 0.23, setting n_components > min(n_features, n_classes - 1) will raise a ValueError. You should set n_components to None (default), or a value smaller or equal to min(n_features, n_classes - 1).
  warnings.warn(future_msg, FutureWarning)
/Users/tedlinghu/opt/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:466: ChangedBehaviorWarning: n_components cannot be larger than min(n_features, n_classes - 1). Using min(n_features, n_classes - 1) = min(289, 2 - 1) = 1 components.
  ChangedBehaviorWarning)
/Users/tedlinghu/opt/anaconda3/lib/python3.7/site-packages/sklearn/di

/Users/tedlinghu/opt/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/Users/tedlinghu/opt/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:466: ChangedBehaviorWarning: n_components cannot be larger than min(n_features, n_classes - 1). Using min(n_features, n_classes - 1) = min(289, 2 - 1) = 1 components.
  ChangedBehaviorWarning)
/Users/tedlinghu/opt/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:472: FutureWarning: In version 0.23, setting n_components > min(n_features, n_classes - 1) will raise a ValueError. You should set n_components to None (default), or a value smaller or equal to min(n_features, n_classes - 1).
  warnings.warn(future_msg, FutureWarning)
/Users/tedlinghu/opt/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear

/Users/tedlinghu/opt/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:466: ChangedBehaviorWarning: n_components cannot be larger than min(n_features, n_classes - 1). Using min(n_features, n_classes - 1) = min(289, 2 - 1) = 1 components.
  ChangedBehaviorWarning)
/Users/tedlinghu/opt/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:472: FutureWarning: In version 0.23, setting n_components > min(n_features, n_classes - 1) will raise a ValueError. You should set n_components to None (default), or a value smaller or equal to min(n_features, n_classes - 1).
  warnings.warn(future_msg, FutureWarning)
/Users/tedlinghu/opt/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/Users/tedlinghu/opt/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:466: ChangedBehaviorWarning: n_components cannot be larger than min(n_features, n_c

In [ ]:
for i in grid_search_dict:
    print(grid_search_dict[i].best_estimator_)

# accuracy from training
lda_upsampled_training': array([0.63414165, 0.62105466, 0.62442263, 0.62769438, 0.63135104]),
 'lda_mult_upsampled_training': array([0.69601617, 0.68321786, 0.67407621, 0.68629715, 0.6895689 ]),
 'lda_div_upsampled_training': array([0.69688222, 0.68812548, 0.68033102, 0.69168591, 0.70294457]),
 'lda_downsampled_training': array([0.6328928 , 0.63529412, 0.64264706, 0.6       , 0.61029412]),
 'lda_mult_downsampled_training': array([0.66519824, 0.68529412, 0.69411765, 0.65588235, 0.65      ]),
 'lda_div_downsampled_training': array([0.6784141 , 0.70735294, 0.71176471, 0.65441176, 0.68382353]),
 
 
 'rf_upsampled_training': array([0.98162048, 0.97786759, 0.97680908, 0.98036952, 0.9806582 ]),
 'rf_mult_upsampled_training': array([0.97911855, 0.97767513, 0.97767513, 0.97902232, 0.98085065]),
 'rf_div_upsampled_training': array([0.97911855, 0.98104311, 0.97690531, 0.97757891, 0.97921478]),
 'rf_downsampled_training': array([0.69016153, 0.675     , 0.65588235, 0.62647059, 0.675     ]),
 'rf_mult_downsampled_training': array([0.68281938, 0.67352941, 0.67058824, 0.64705882, 0.66470588]),
 'rf_div_downsampled_training': array([0.6681351 , 0.68088235, 0.67205882, 0.62352941, 0.64705882])}
 
 

# ROC  from training
'lda_upsampled_training': array([0.68180315, 0.6650475 , 0.6670451 , 0.6724549 , 0.67714269]),
 'lda_mult_upsampled_training': array([0.75557933, 0.74550103, 0.73825109, 0.74636538, 0.75530968]),
 'lda_div_upsampled_training': array([0.76700429, 0.75786461, 0.75280098, 0.76169017, 0.77274111]),
 'lda_downsampled_training': array([0.69616181, 0.68557958, 0.67672145, 0.63568339, 0.66915225]),
 'lda_mult_downsampled_training': array([0.72609971, 0.74940311, 0.73609862, 0.68984429, 0.71172145]),
 'lda_div_downsampled_training': array([0.75320856, 0.77116782, 0.77732699, 0.70757785, 0.74186851]),
 
  'rf_upsampled_training': array([0.99999211, 0.99999059, 0.99998318, 0.9999582 , 0.99987451]),
 'rf_mult_upsampled_training': array([0.99999585, 0.99999504, 0.99986379, 0.99998424, 0.99999293]),
 'rf_div_upsampled_training': array([0.99968596, 0.99989094, 0.99978702, 0.99994544, 0.99979247]),
 'rf_downsampled_training': array([0.72484043, 0.73517301, 0.72260813, 0.70577855, 0.71755623]),
 'rf_mult_downsampled_training': array([0.72140331, 0.73644031, 0.74752595, 0.70022059, 0.70479671]),
 'rf_div_downsampled_training': array([0.72367173, 0.72952422, 0.72228374, 0.70492647, 0.73513408])}

# F1 from training

'lda_upsampled_training': array([0.63051506, 0.61907526, 0.62416948, 0.628231  , 0.63067579]),
 'lda_mult_upsampled_training': array([0.70262638, 0.69135571, 0.68110347, 0.69464219, 0.6970892 ]),
 'lda_div_upsampled_training': array([0.70773798, 0.70021275, 0.691436  , 0.70316843, 0.71376912]),
 'lda_downsampled_training': array([0.62908012, 0.63953488, 0.64421669, 0.58912387, 0.59665145]),
 'lda_mult_downsampled_training': array([0.68333333, 0.69602273, 0.7037037 , 0.6568915 , 0.64792899]),
 'lda_div_downsampled_training': array([0.69876204, 0.71692745, 0.7223796 , 0.66284075, 0.68613139]),
 
 'rf_upsampled_training': array([0.98087673, 0.98058062, 0.97847243, 0.97869834, 0.97927512]),
 'rf_mult_upsampled_training': array([0.97907457, 0.97746728, 0.97456123, 0.97766847, 0.978372  ]),
 'rf_div_upsampled_training': array([0.97605675, 0.97706467, 0.97514793, 0.97756789, 0.98107658]),
 'rf_downsampled_training': array([0.64353312, 0.6519685 , 0.68847352, 0.61341853, 0.66666667]),
 'rf_mult_downsampled_training': array([0.65443425, 0.63223787, 0.67088608, 0.60547504, 0.6365105 ]),
 'rf_div_downsampled_training': array([0.65432099, 0.65648855, 0.66144201, 0.6192733 , 0.63768116])}

In [9]:
cvres = grid_search.cv_results_

for mean_score in cvres['mean_test_score']:
    print(mean_score)

0.6041747782825005
0.7016655851178888
0.7223015357992646
0.6461388708630759
0.7135409906986804
0.745014060134112
0.6484317542721176
0.7025308241401687
0.7239671209171534
0.6432403201384383
0.6958901146441703
0.7316461172398876
0.6342850962578414
0.6952844473285745
0.6119186675319056
0.7147739563054295
0.664070949599827
0.6895089768548561


In [ ]:
0.6024305206084036
0.5930580588059835
0.6420007064472099
0.6098641826462903
0.5732782736280219
0.6347595820168533
0.6077369446852032
0.6122187487085178
0.6611368386351524
0.6227445375882699
0.5967225171919269
0.6349226058076906
0.5950537333898259
0.5782709672567685
0.6033953109884425
0.5994295798408042
0.5798160004940426
0.605694765089247